In [170]:
import pandas as pd
import numpy as np
from numpy import mean
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve

In [171]:
df = pd.read_csv("/content/drive/MyDrive/College Project/Bank Churners/BankChurners.csv")
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [172]:
# drop the customer id's
#df = df.drop(["CLIENTNUM","Dependent_count"],axis = 1)
df = df.drop(["CLIENTNUM"],axis = 1)

In [173]:
df.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [174]:
df.dtypes

Attrition_Flag               object
Customer_Age                  int64
Gender                       object
Dependent_count               int64
Education_Level              object
Marital_Status               object
Income_Category              object
Card_Category                object
Months_on_book                int64
Total_Relationship_Count      int64
Months_Inactive_12_mon        int64
Contacts_Count_12_mon         int64
Credit_Limit                float64
Total_Revolving_Bal           int64
Avg_Open_To_Buy             float64
Total_Amt_Chng_Q4_Q1        float64
Total_Trans_Amt               int64
Total_Trans_Ct                int64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
dtype: object

In [175]:
print("Size of Dataset: {} rows , {} columns".format(df.shape[0],df.shape[1]))

Size of Dataset: 10127 rows , 20 columns


In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  object 
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

In [177]:
df.describe()

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
count,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000
mean,46.325960,2.346203,35.928409,3.812580,2.341167,2.455317,8631.953698,1162.814061,7469.139637,0.759941,4404.086304,64.858695,0.712222,0.274894
std,8.016814,1.298908,7.986416,1.554408,1.010622,1.106225,9088.776650,814.987335,9090.685324,0.219207,3397.129254,23.472570,0.238086,0.275691
min,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000
25%,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,359.000000,1324.500000,0.631000,2155.500000,45.000000,0.582000,0.023000
50%,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1276.000000,3474.000000,0.736000,3899.000000,67.000000,0.702000,0.176000
75%,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,11067.500000,1784.000000,9859.000000,0.859000,4741.000000,81.000000,0.818000,0.503000
max,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,18484.000000,139.000000,3.714000,0.999000


In [178]:
#checking for missing values

def missing (df):
    missing_number = df.isnull().sum().sort_values(ascending=False)
    missing_percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Missing_Number', 'Missing_Percent'])
    return missing_values

missing(df)

,Missing_Number,Missing_Percent
Attrition_Flag,0,0.0
Customer_Age,0,0.0
Total_Ct_Chng_Q4_Q1,0,0.0
Total_Trans_Ct,0,0.0
Total_Trans_Amt,0,0.0
Total_Amt_Chng_Q4_Q1,0,0.0
Avg_Open_To_Buy,0,0.0
Total_Revolving_Bal,0,0.0
Credit_Limit,0,0.0
Contacts_Count_12_mon,0,0.0


#checking for outliers

In [179]:
def checkOutliers(data,Title):
  fig = px.box(df,
               x=data,
               notched=True)
  fig.update_layout(
      title={
          'text': Title,
          'y':0.95,
          'x':0.45,
          'xanchor': 'center',
          'yanchor': 'top'})
  fig.show()

In [182]:
# def remove_outlier(df_in, col_name):
#     q1 = df_in[col_name].quantile(0.25)
#     q3 = df_in[col_name].quantile(0.75)
#     iqr = q3-q1 #Interquartile range
#     fence_low  = q1-1.5*iqr
#     fence_high = q3+1.5*iqr
#     df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
#     return df_out

def drop_numerical_outliers(df, z_thresh=3):
    # Constrains will contain `True` or `False` depending on if it is a value below the threshold.
    constrains = df.select_dtypes(include=[np.number]).apply(lambda x: np.abs(stats.zscore(x)) < z_thresh, reduce=False) \
        .all(axis=1)
    # Drop (inplace) values set to be rejected
    df.drop(df.index[~constrains], inplace=True)

drop_numerical_outliers(df)

TypeError: ignored

In [ ]:
checkOutliers("Customer_Age","Customer Age")

In [ ]:
# df = remove_outlier(df,"Customer_Age")
# checkOutliers("Customer_Age","Customer Age After Outlier Treatment")

In [ ]:
checkOutliers("Months_on_book","Period of relationship with bank")

In [ ]:
# df = remove_outlier(df,"Months_on_book")
# checkOutliers("Months_on_book","Period of relationship with bank After Outlier Treatment")

In [ ]:
checkOutliers("Total_Relationship_Count","Total no. of products held by the customer")

In [ ]:
checkOutliers("Months_Inactive_12_mon","No. of months inactive in the last 12 months")

In [ ]:
# df = remove_outlier(df,"Months_Inactive_12_mon")
# checkOutliers("Months_Inactive_12_mon","No. of months inactive in the last 12 months After Outlier Treatment")

In [ ]:
checkOutliers("Contacts_Count_12_mon","No. of Contacts in the last 12 months")

In [ ]:
# df = remove_outlier(df,"Contacts_Count_12_mon")
# checkOutliers("Contacts_Count_12_mon","No. of Contacts in the last 12 months After Outlier Treatment")

In [ ]:
checkOutliers("Credit_Limit","Credit Limit on the Credit Card")

In [ ]:
# df = remove_outlier(df,"Credit_Limit")
# checkOutliers("Credit_Limit","Credit Limit on the Credit Card After Outlier Treatment")

In [ ]:
checkOutliers("Total_Revolving_Bal","Total Revolving Balance on the Credit Card")

In [ ]:
checkOutliers("Avg_Open_To_Buy","Open to Buy Credit Line (Average of last 12 months)")

In [ ]:
# df = remove_outlier(df,"Avg_Open_To_Buy")
# checkOutliers("Avg_Open_To_Buy","Open to Buy Credit Line (Average of last 12 months)  After Outlier Treatment")

In [ ]:
checkOutliers("Total_Amt_Chng_Q4_Q1","Change in Transaction Amount (Q4 over Q1)")

In [ ]:
# df = remove_outlier(df,"Total_Amt_Chng_Q4_Q1")
# checkOutliers("Total_Amt_Chng_Q4_Q1","Change in Transaction Amount (Q4 over Q1) After Outlier Treatment")

In [ ]:
checkOutliers("Total_Trans_Amt","Total Transaction Amount (Last 12 months)")

In [ ]:
# df = remove_outlier(df,"Total_Trans_Amt")
# checkOutliers("Total_Trans_Amt","Total Transaction Amount (Last 12 months) After Outlier Treatment")

In [ ]:
checkOutliers("Total_Trans_Ct","Total Transaction Count (Last 12 months)")

In [ ]:
# df = remove_outlier(df,"Total_Trans_Ct")
# checkOutliers("Total_Trans_Ct","Total Transaction Count (Last 12 months)")

In [ ]:
checkOutliers("Avg_Utilization_Ratio","Average Card Utilization Ratio")

In [ ]:
print("Size of New Dataset: {} rows , {} columns".format(df.shape[0],df.shape[1]))

In [ ]:
def numericDataVisualization(x_name,colorValue,Title):
  fig = px.histogram(df,
                     x = x_name,
                     color = colorValue,
                     marginal = "violin", # or box, rug
                     )
  fig.update_layout(
    title={
        'text': Title,
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})
  
  fig.show()
  print("Skewness: %f" % df[x_name].skew())
  print("Kurtosis: %f" % df[x_name].kurt())

Data visualization with respect to Attrition_Flag Column

In [ ]:
numericDataVisualization("Customer_Age","Attrition_Flag","Customer Age")

In [ ]:
numericDataVisualization("Months_on_book","Attrition_Flag","Period of relationship with bank")

In [ ]:
numericDataVisualization("Total_Relationship_Count","Attrition_Flag","Total no. of products held by the customer")

In [ ]:
numericDataVisualization("Months_Inactive_12_mon","Attrition_Flag","No. of months inactive in the last 12 months")

In [ ]:
numericDataVisualization("Contacts_Count_12_mon","Attrition_Flag","No. of Contacts in the last 12 months")

In [ ]:
numericDataVisualization("Credit_Limit","Attrition_Flag","Credit Limit on the Credit Card")

In [ ]:
numericDataVisualization("Total_Revolving_Bal","Attrition_Flag","Total Revolving Balance on the Credit Card")

In [ ]:
numericDataVisualization("Avg_Open_To_Buy","Attrition_Flag","Open to Buy Credit Line (Average of last 12 months)")

In [ ]:
numericDataVisualization("Total_Amt_Chng_Q4_Q1","Attrition_Flag","Change in Transaction Amount (Q4 over Q1)")

In [ ]:
numericDataVisualization("Total_Trans_Amt","Attrition_Flag","Total Transaction Amount (Last 12 months)")

In [ ]:
numericDataVisualization("Total_Trans_Ct","Attrition_Flag","Total Transaction Count (Last 12 months)")

In [ ]:
numericDataVisualization("Total_Ct_Chng_Q4_Q1","Attrition_Flag","Change in Transaction Count (Q4 over Q1)")

In [ ]:
numericDataVisualization("Avg_Utilization_Ratio","Attrition_Flag","Average Card Utilization Ratio")

In [ ]:
def categoricalDataVisualization(x_name,colorName,Title):
  fig = px.histogram(df,
                     x=x_name,
                     color = colorName,
                     text_auto=True).update_xaxes(categoryorder = "total descending")
  fig.update_layout(
      title={
          'text': Title,
          'y':0.95,
          'x':0.45,
          'xanchor': 'center',
          'yanchor': 'top'})
  fig.show()

In [ ]:
categoricalDataVisualization("Education_Level","Attrition_Flag","Education Level")

In [ ]:
categoricalDataVisualization("Marital_Status","Attrition_Flag","Education Level")

In [ ]:
categoricalDataVisualization("Income_Category","Attrition_Flag","Education Level")

In [ ]:
categoricalDataVisualization("Card_Category","Attrition_Flag","Education Level")

In [ ]:
categoricalDataVisualization("Attrition_Flag","Gender","Checking For Imbalanced Data")

**Our dataset contains imbalanced data**

In [ ]:
df.Attrition_Flag = df.Attrition_Flag.replace({'Attrited Customer':1,'Existing Customer':0})
df.Gender = df.Gender.replace({'F':1,'M':0})
df = pd.concat([df,pd.get_dummies(df['Education_Level']).drop(columns=['Unknown'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['Income_Category']).drop(columns=['Unknown'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['Marital_Status']).drop(columns=['Unknown'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['Card_Category']).drop(columns=['Platinum'])],axis=1)
df.drop(columns = ['Education_Level','Income_Category','Marital_Status','Card_Category'],inplace=True)

In [ ]:
req_cols = ['Customer_Age', 'Months_on_book', 'Total_Relationship_Count',
            'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
            'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1',
            'Avg_Utilization_Ratio']
            
scaler = StandardScaler()
df[req_cols] = scaler.fit_transform(df[req_cols])

In [ ]:
style.use('ggplot')
sns.set_style('whitegrid')
plt.subplots(figsize = (30,30))
## Plotting heatmap. Generate a mask for the upper triangle (taken from seaborn example gallery)
sns.heatmap(df.corr(), cmap=sns.diverging_palette(20, 220, n=200), annot=True, center = 0)
plt.title("Heatmap of all the Features of data", fontsize = 25)
plt.show()

In [ ]:
y = df.pop("Attrition_Flag")
x = df

In [ ]:
x.head()

In [ ]:
sm = SMOTE(random_state = 69, sampling_strategy = 1.0)
x, y = sm.fit_resample(x, y)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(x,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=x.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
feat_importances.nlargest(10)

In [ ]:
Top10Features = ['Total_Trans_Ct',
                 'Total_Trans_Amt',
                 'Total_Revolving_Bal',
                 'Months_Inactive_12_mon',
                 'Total_Ct_Chng_Q4_Q1',
                 'Total_Relationship_Count',
                 'Avg_Utilization_Ratio',
                 'Total_Amt_Chng_Q4_Q1',
                 'Credit_Limit',
                 'Contacts_Count_12_mon']
x = x[Top10Features]
x.head()

In [ ]:
# x_pca = PCA(n_components = 31, random_state = 410)
# x_pca.fit(x)
# x_pca_31 = x_pca.transform(x)

In [ ]:
# x_pca_31.shape

In [ ]:
# plt.plot(np.cumsum(x_pca.explained_variance_ratio_))
# plt.xlabel("Number of Components")
# plt.ylabel("Explained Variance")
# plt.show()

In [ ]:
# print("Variance Explained by each components")
# for i in range(0,31):
#   print("Variance Explained by first {} components : {}".format(i,np.cumsum(x_pca.explained_variance_ratio_)[i]))

In [ ]:
# x_pca = PCA(n_components = 8, random_state = 410)
# x_pca.fit(x)
# x_pca_8 = x_pca.transform(x)
# x_pca_8

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.75,stratify = y, random_state = 42)

In [ ]:
cv = KFold(n_splits = 5, random_state=100, shuffle=True)

LrModel = LogisticRegression()
scores = cross_val_score(LrModel, X_train, y_train, cv = cv,scoring='accuracy')
print("Logistics Regression Average CV Score: ",scores.mean())

KnModel = KNeighborsClassifier()
scores = cross_val_score(KnModel, X_train, y_train, cv = cv,scoring='accuracy')
print("KNeighbors Average CV Score: ",scores.mean())

DtModel = DecisionTreeClassifier()
scores = cross_val_score(DtModel, X_train, y_train, cv = cv,scoring='accuracy')
print("Decision Tree Average CV Score: ",scores.mean())

RfModel = RandomForestClassifier()
scores = cross_val_score(RfModel, X_train, y_train, cv = cv,scoring='accuracy')
print("Random Forest Average CV Score: ",scores.mean())

AbModel = AdaBoostClassifier()
scores = cross_val_score(AbModel, X_train, y_train, cv = cv,scoring='accuracy')
print("AdaBoost Tree Average CV Score: ",scores.mean())

SvmModel = SVC(kernel= 'linear', random_state=1, C=0.1)
scores = cross_val_score(SvmModel, X_train, y_train, cv = cv,scoring='accuracy')
print("Support Vector Machine Average CV Score: ",scores.mean())

In [ ]:
# decision Tree 
rf_params = {
    'max_features': ['sqrt','auto','log2',None],
    'max_depth': sp_randint(5,50),
    'min_samples_leaf': sp_randint(1,15),
    'min_samples_split': sp_randint(2,30),
    'criterion':['gini','entropy'],
    'random_state': [100]
    }
n_iter_search=100

# for k in rf_params:
#     n_iter_search=round(n_iter_search*len(rf_params[k]))
# n_iter_search=n_iter_search*0.25

Random = RandomizedSearchCV(DtModel, param_distributions=rf_params,n_iter=n_iter_search,cv=cv,scoring='accuracy')
Random.fit(X_train, y_train)
print(Random.best_params_)
print('Accuracy: %.5f (%.5f)' % (Random.best_score_*100, mean(Random.cv_results_['std_test_score'])*100))

In [ ]:
# KNeighour 
rf_params = {
    'n_neighbors': sp_randint(1,20),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

n_iter_search=50

Random = RandomizedSearchCV(KnModel, param_distributions=rf_params,n_iter=n_iter_search,cv=cv,scoring='accuracy')
Random.fit(X_train, y_train)

print(Random.best_params_)
print('Accuracy: %.5f (%.5f)' % (Random.best_score_*100, mean(Random.cv_results_['std_test_score'])*100))

In [ ]:
rf_params = {
    'penalty' : [ 'l1', 'l2' , 'elasticnet'],
    'C' : stats.uniform(0,1),
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'random_state': [100]
    #'class_weight': ['balanced', None ]
} #150

n_iter_search=50

Random = RandomizedSearchCV(LrModel, param_distributions=rf_params,n_iter=n_iter_search,cv=cv,scoring='accuracy')
Random.fit(X_train, y_train)

print(Random.best_params_)
print('Accuracy: %.5f (%.5f)' % (Random.best_score_*100, mean(Random.cv_results_['std_test_score'])*100))

In [ ]:
# rf_params = {
#     'n_estimators': sp_randint(5,500),
#     'max_features': ['sqrt','auto','log2'],
#     'max_depth': sp_randint(5,50),
#     'min_samples_leaf': sp_randint(1,15),
#     'min_samples_split':sp_randint(2,30),
#     'criterion':['gini','entropy'],
#     'random_state': [100]
# } 

# n_iter_search=300
# # for k in rf_params:
# #     n_iter_search=round(n_iter_search*len(rf_params[k]))


# # n_iter_search=n_iter_search*0.25

# Random = RandomizedSearchCV(RfModel, param_distributions=rf_params,n_iter=n_iter_search,cv=cv,scoring='accuracy')
# Random.fit(X_train, y_train)

# print(Random.best_params_)
# print('Accuracy: %.5f (%.5f)' % (Random.best_score_*100, mean(Random.cv_results_['std_test_score'])*100))

In [ ]:
model= SVC()
param_grid = {'kernel': ['rbf','linear'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}

modelsvm = RandomizedSearchCV(SvmModel,param_distributions = param_grid, cv=5, scoring="accuracy", n_jobs= 4, verbose = 1)

modelsvm.fit(X_train,y_train)

# Best score
print(modelsvm.best_params_)
print('Accuracy: %.5f (%.5f)' % (Random.best_score_*100, mean(Random.cv_results_['std_test_score'])*100))

In [ ]:
cv = KFold(n_splits = 5, random_state=100, shuffle=True)

LrModel = LogisticRegression(C = 0.20164760646072755, penalty = 'l2', random_state = 100, solver = 'liblinear')
scores = cross_val_score(LrModel, X_train, y_train, cv = cv,scoring='accuracy')
print("Logistics Regression Average CV Score: ",scores.mean())

KnModel = KNeighborsClassifier(algorithm = 'brute', n_neighbors = 1, weights = 'uniform')
scores = cross_val_score(KnModel, X_train, y_train, cv = cv,scoring='accuracy')
print("KNeighbors Average CV Score: ",scores.mean())

DtModel = DecisionTreeClassifier(criterion = 'gini', max_depth = 44, max_features = None, min_samples_leaf = 4, min_samples_split = 2, random_state = 100)
scores = cross_val_score(DtModel, X_train, y_train, cv = cv,scoring='accuracy')
print("Decision Tree Average CV Score: ",scores.mean())

RfModel = RandomForestClassifier(criterion = 'entropy', max_depth = 26, max_features = 'auto', min_samples_leaf = 9, min_samples_split = 18, n_estimators = 40, random_state = 100)
scores = cross_val_score(RfModel, X_train, y_train, cv = cv,scoring='accuracy')
print("Random Forest Average CV Score: ",scores.mean())

AbModel = AdaBoostClassifier()
scores = cross_val_score(AbModel, X_train, y_train, cv = cv,scoring='accuracy')
print("AdaBoost Tree Average CV Score: ",scores.mean())

SvmModel = SVC(kernel= 'linear', gamma = 1, C=1)
scores = cross_val_score(SvmModel, X_train, y_train, cv = cv,scoring='accuracy')
print("Support Vector Machine Average CV Score: ",scores.mean())

In [ ]:
KnModel.fit(X_train,y_train)
prediction = KnModel.predict(X_test)
print("The Accuracy Score on train data:", accuracy_score(KnModel.predict(X_train),y_train))
print("The Accuracy Score on test data:", accuracy_score(prediction,y_test))
print("The precision Score :", precision_score(prediction,y_test))
print("The recall Score :", recall_score(prediction,y_test))
print("The f1 score :", f1_score(prediction,y_test))
print("The Classification report : \n",classification_report(prediction,y_test))
sns.heatmap(confusion_matrix(prediction,y_test), annot = True)
plt.show()

In [ ]:
DtModel.fit(X_train,y_train)
prediction = DtModel.predict(X_test)
print("The Accuracy Score on train data:", accuracy_score(DtModel.predict(X_train),y_train))
print("The Accuracy Score on test data:", accuracy_score(prediction,y_test))
print("The precision Score :", precision_score(prediction,y_test))
print("The recall Score :", recall_score(prediction,y_test))
print("The f1 score :", f1_score(prediction,y_test))
print("The Classification report : \n",classification_report(prediction,y_test))
sns.heatmap(confusion_matrix(prediction,y_test), annot = True)
plt.show()

In [ ]:
RfModel.fit(X_train,y_train)
prediction = RfModel.predict(X_test)
print("The Accuracy Score on train data:", accuracy_score(RfModel.predict(X_train),y_train))
print("The Accuracy Score on test data:", accuracy_score(prediction,y_test))
print("The precision Score :", precision_score(prediction,y_test))
print("The recall Score :", recall_score(prediction,y_test))
print("The f1 score :", f1_score(prediction,y_test))
print("The Classification report : \n",classification_report(prediction,y_test))
sns.heatmap(confusion_matrix(prediction,y_test), annot = True)
plt.show()

In [ ]:
AbModel = AdaBoostClassifier()
AbModel.fit(X_train,y_train)
prediction = AbModel.predict(X_test)
print("The Accuracy Score on train data:", accuracy_score(AbModel.predict(X_train),y_train))
print("The Accuracy Score on test data:", accuracy_score(prediction,y_test))
print("The precision Score :", precision_score(prediction,y_test))
print("The recall Score :", recall_score(prediction,y_test))
print("The f1 score :", f1_score(prediction,y_test))
print("The Classification report : \n",classification_report(prediction,y_test))
sns.heatmap(confusion_matrix(prediction,y_test), annot = True)
plt.show()